In [1]:
! nvidia-smi

Mon Jun 20 18:57:15 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    25W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# ! gdown --id 1PUSJ56k93B42XW9oszcPDefM8HeJ2BOg
# ! pip -q uninstall -y kaggle
# ! pip -q install --upgrade pip
# ! pip -q install kaggle --upgrade
# ! mkdir ~/.kaggle
# ! cp kaggle.json ~/.kaggle/
# ! chmod 600 ~/.kaggle/kaggle.json
# ! kaggle competitions download us-patent-phrase-to-phrase-matching
# ! kaggle datasets download yasufuminakama/cpc-data
# ! kaggle datasets download aishikai/uspppm-weights-1
# ! kaggle datasets download aishikai/uspppm-weights-2
# ! kaggle datasets download aishikai/fasttext-word-embeddings

In [4]:
# ! unzip -q /content/us-patent-phrase-to-phrase-matching.zip -d data
# ! rm /content/us-patent-phrase-to-phrase-matching.zip
# ! unzip -q /content/cpc-data.zip -d cpc_data
# ! rm /content/cpc-data.zip
# ! unzip -q /content/uspppm-weights-1.zip -d uspppm-weights-1
# ! rm /content/uspppm-weights-1.zip
# ! unzip -q /content/uspppm-weights-2.zip -d uspppm-weights-2
# ! rm /content/uspppm-weights-2.zip
# ! unzip /content/fasttext-word-embeddings.zip
# ! rm /content/fasttext-word-embeddings.zip
# ! unzip -q /content/drive/MyDrive/USPPM-deberta-v3-large-PT-BCC-MSE-1-133-tags.zip -d /content/uspppm-weights-3

In [5]:
# !git clone --recursive https://github.com/Microsoft/LightGBM

In [6]:
%cd /content/LightGBM

/content/LightGBM


In [7]:
!mkdir build
!cmake -DUSE_GPU=1 #avoid ..
!make -j$(nproc)
!sudo apt-get -y install python-pip
!sudo -H pip install setuptools pandas numpy scipy scikit-learn -U

mkdir: cannot create directory ‘build’: File exists
CMake Warning:
  No source or binary directory provided.  Both will be assumed to be the
  same as the current working directory, but note that this warning will
  become a fatal error in future CMake releases.


-- OpenCL include directory: /usr/include
-- Using _mm_prefetch
-- Using _mm_malloc
-- Configuring done
-- Generating done
-- Build files have been written to: /content/LightGBM
Consolidate compiler generated dependencies of target lightgbm_capi_objs
[  2%] Built target lightgbm_capi_objs
Consolidate compiler generated dependencies of target lightgbm_objs
[ 92%] Built target lightgbm_objs
Consolidate compiler generated dependencies of target lightgbm
[ 94%] Built target _lightgbm
[100%] Built target lightgbm
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-pip is already the newest version (9.0.1-2.3~ubuntu1.18.04.5).
The following package was automatically installed and i

In [8]:
%cd /content/LightGBM/python-package

/content/LightGBM/python-package


In [9]:
!sudo python setup.py install --precompile

running install
/usr/local/lib/python3.7/dist-packages/setuptools/command/install.py:37: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  setuptools.SetuptoolsDeprecationWarning,
running build
running build_py
running egg_info
writing lightgbm.egg-info/PKG-INFO
writing dependency_links to lightgbm.egg-info/dependency_links.txt
writing requirements to lightgbm.egg-info/requires.txt
writing top-level names to lightgbm.egg-info/top_level.txt
reading manifest template 'MANIFEST.in'
no previously-included directories found matching 'build'
writing manifest file 'lightgbm.egg-info/SOURCES.txt'
copying lightgbm/VERSION.txt -> build/lib/lightgbm
running install_lib
copying build/lib/lightgbm/VERSION.txt -> /usr/lib/python3.7/site-packages/lightgbm
INFO:LightGBM:Installing lib_lightgbm from: ['/content/LightGBM/lib_lightgbm.so']
running install_egg_info
removing '/usr/lib/python3.7/site-packages/lightgbm-3.3.2.99-py3.7.egg-info' 

In [10]:
# ! pip -q install fasttext
# ! pip -q install optuna

In [11]:
import os
import random
import numpy as np
import pandas as pd
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GroupKFold , StratifiedKFold
from sklearn.model_selection import  KFold
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy as sp
import glob
from tqdm.notebook import tqdm 

import fasttext as ft
import lightgbm as lgbm
from xgboost import XGBRegressor

import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler

import warnings
warnings.filterwarnings("ignore")

In [12]:
% cd ..

/content/LightGBM


In [13]:
% cd ..

/content


In [14]:
class CFG:
    seed = 42
    num_folds = 5
    trn_fold = [0, 1, 2, 3, 4]

In [15]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
seed_everything(seed=CFG.seed)

In [16]:
def get_folds(df):
    df['score_map'] = df['score'].map({0.00: 0, 0.25: 1, 0.50: 2, 0.75: 3, 1.00: 4})
    encoder = LabelEncoder()
    df['anchor_map'] = encoder.fit_transform(df['anchor'])
    Fold = StratifiedKFold(n_splits=CFG.num_folds, shuffle=True, random_state=CFG.seed)
    for n, (df_index, val_index) in enumerate(Fold.split(df, df['score_map'], groups = df['anchor_map'])):
        df.loc[val_index, 'fold'] = int(n)
    df['fold'] = df['fold'].astype(int)
    return df

In [17]:
def get_cpc_texts():
    contexts = []
    pattern = '[A-Z]\d+'
    for file_name in os.listdir('/content/cpc_data/CPCSchemeXML202105'):
        result = re.findall(pattern, file_name)
        if result:
            contexts.append(result)
    contexts = sorted(set(sum(contexts, [])))
    results = {}
    for cpc in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'Y']:
        with open(f'/content/cpc_data/CPCTitleList202202/cpc-section-{cpc}_20220201.txt') as f:
            s = f.read()
        pattern = f'{cpc}\t\t.+'
        result = re.findall(pattern, s)
        cpc_result = result[0].lstrip(pattern)
        for context in [c for c in contexts if c[0] == cpc]:
            pattern = f'{context}\t\t.+'
            result = re.findall(pattern, s)
            results[context] = cpc_result + ". " + result[0].lstrip(pattern)
    return results

In [18]:
def scoring(y_true, y_pred):
    score = sp.stats.pearsonr(y_true, y_pred)[0]
    return score

In [19]:
encoder = LabelEncoder()

In [20]:
train = pd.read_csv("/content/data/train.csv")
cpc_texts = get_cpc_texts()
train['context_text'] = train['context'].map(cpc_texts)
train["context_tags"] = '<' + train['context'] + '>'
train['context_num'] = encoder.fit_transform(train['context'])
# train['text'] = train['anchor'] + ' ' + train['target'] + ' '  + train['context_text']
train['text'] = train['anchor'] + ' ' + train['target']
train['text'] = train['text'].map(lambda x: x.lower())
train = get_folds(train)

In [21]:
model_dict = {
    "USPPM-deberta-v3-large-BCC-MSE-2-133-tags" : {
        'config' : '../input/huggingface-models/deberta-v3-large/config.json',
        'tokenizer' : '../input/uspppm-weights-1/USPPM-deberta-v3-large-BCC-MSE-2-133-tags/models/tokenizer/',
        'model_weights' : [f'../input/uspppm-weights-1/USPPM-deberta-v3-large-BCC-MSE-2-133-tags/models/model_{fold}.pth' for fold in range(5)],
        'fold_weights' : [0.86554,0.85818,0.85701,0.85384,0.86284],
        'max_len' : 133,
        'version' : 2,
        'directory': 1
    },
    "USPPM-deberta-v3-large-XE-NA-2.1-133-tags" : {
        'config' : '../input/huggingface-models/deberta-v3-large/config.json',
        'tokenizer' : '../input/uspppm-weights-1/USPPM-deberta-v3-large-XE-NA-2.1-133-tags/models/tokenizer/',
        'model_weights' : [f'../input/uspppm-weights-1/USPPM-deberta-v3-large-XE-NA-2.1-133-tags/models/model_{fold}.pth' for fold in range(5)],
        'fold_weights' : [0.86811,0.85894,0.86076,0.85937,0.86524],
        'max_len' : 133,
        'version' : 2.1,
        'directory': 1
    },
    "USPPM-bert-for-patents-BCC-MSE-2-117-tags" : {
        'config' : '../input/huggingface-models/bert-for-patents/config.json',
        'tokenizer' : '../input/uspppm-weights-1/USPPM-bert-for-patents-BCC-MSE-2-117-tags/models/tokenizer/',
        'model_weights' : [f'../input/uspppm-weights-1/USPPM-bert-for-patents-BCC-MSE-2-117-tags/models/model_{fold}.pth' for fold in range(5)],
        'fold_weights' : [0.85571,0.84781,0.85376,0.84724,0.85582],
        'max_len' : 117,
        'version' : 2,
        'directory': 1
    },
    "USPPM-bert-for-patents-XE-NA-2.1-117-tags" : {
        'config' : '../input/huggingface-models/bert-for-patents/config.json',
        'tokenizer' : '../input/uspppm-weights-1/USPPM-bert-for-patents-XE-NA-2.1-117-tags/models/tokenizer/',
        'model_weights' : [f'../input/uspppm-weights-1/USPPM-bert-for-patents-XE-NA-2.1-117-tags/models/model_{fold}.pth' for fold in range(5)],
        'fold_weights' : [0.86499,0.85723,0.86173,0.86141,0.85525],
        'max_len' : 117,
        'version' : 2.1,
        'directory': 1
    },
#     "USPPM-PatentSBERTa-XE-NA-2.1-125-tags": {
#         'config' : '../input/huggingface-models/PatentSBERTa/config.json',
#         'tokenizer' : '../input/uspppm-weights-1/USPPM-PatentSBERTa-XE-NA-2.1-125-tags/models/tokenizer/',
#         'model_weights' : [f'../input/uspppm-weights-1/USPPM-PatentSBERTa-XE-NA-2.1-125-tags/models/model_{fold}.pth' for fold in range(5)],
#         'fold_weights' : [0.83789,0.8342,0.82875,0.82359,0.82535],
#         'max_len' : 125,
#         'version' : 2.1,
#         'directory': 1
#     },
#     "USPPM-deberta-v3-base-BCC-MSE-1-133-tags": {
#         'config' : '../input/huggingface-models/deberta-v3-base/config.json',
#         'tokenizer' : '../input/uspppm-weights-1/USPPM-deberta-v3-base-BCC-MSE-1-133-tags/models/tokenizer/',
#         'model_weights' : [f'../input/uspppm-weights-1/USPPM-deberta-v3-base-BCC-MSE-1-133-tags/models/fold_{fold}/model.pth' for fold in range(5)],
#         'fold_weights' : [0.83789,0.8342,0.82875,0.82359,0.82535],
#         'max_len' : 133,
#         'version' : 1,
#         'directory': 1
#     },
    "USPPM-deberta-v3-base-XE-NA-2.1-133-tags": {
        'config' : '../input/huggingface-models/deberta-v3-base/config.json',
        'tokenizer' : '../input/uspppm-weights-1/USPPM-deberta-v3-base-XE-NA-2.1-133-tags/models/tokenizer/',
        'model_weights' : [f'../input/uspppm-weights-1/USPPM-deberta-v3-base-XE-NA-2.1-133-tags/models/model_{fold}.pth' for fold in range(5)],
        'fold_weights' : [0.85803,0.845,0.84902,0.84866,0.85141],
        'max_len' : 133,
        'version' : 2.1,
        'directory': 1
    },
#     "USPPM-simcse-bert-for-patent-XE-NA-2.1-175-tags": {
#         'config' : '../input/huggingface-models/simcse-bert-for-patents/config.json',
#         'tokenizer' : '../input/uspppm-weights-1/USPPM-simcse-bert-for-patent-XE-NA-2.1-175-tags/models/tokenizer/',
#         'model_weights' : [f'../input/uspppm-weights-1/USPPM-simcse-bert-for-patent-XE-NA-2.1-175-tags/models/model_{fold}.pth' for fold in range(5)],
#         'fold_weights' : [0.84609,0.85297,0.8382,0.84458,0.8458],
#         'max_len' : 175,
#         'version' : 2.1,
#         'directory': 1
#     },
#     "USPPM-deberta-v3-small-BCC-MSE-2-133-tags": {
#         'config' : '../input/huggingface-models/deberta-v3-small/config.json',
#         'tokenizer' : '../input/uspppm-weights-2/USPPM-deberta-v3-small-BCC-MSE-2-133-tags/models/tokenizer/',
#         'model_weights' : [f'../input/uspppm-weights-2/USPPM-deberta-v3-small-BCC-MSE-2-133-tags/models/model_{fold}.pth' for fold in range(5)],
#         'fold_weights' : [0.83849,0.83638,0.82693,0.8288,0.8329],
#         'max_len' : 133,
#         'version' : 2,
#         'directory' ; 2
#     },
#     "USPPM-deberta-v3-small-XE-NA-2.1-133-tags": {
#         'config' : '../input/huggingface-models/deberta-v3-small/config.json',
#         'tokenizer' : '../input/uspppm-weights-2/USPPM-deberta-v3-small-XE-NA-2.1-133-tags/models/tokenizer/',
#         'model_weights' : [f'../input/uspppm-weights-2/USPPM-deberta-v3-small-XE-NA-2.1-133-tags/models/model_{fold}.pth' for fold in range(5)],
#         'fold_weights' : [0.85025,0.84183,0.84166,0.8377,0.84328],
#         'max_len' : 133,
#         'version' : 2.1,
#         'directory' ; 2
#     },
    "USPPM-deberta-large-mnli-XE-NA-2.1-175-tags": {
        'config' : '../input/huggingface-models/deberta-large-mnli/config.json',
        'tokenizer' : '../input/uspppm-weights-2/USPPM-deberta-large-mnli-XE-NA-2.1-175-tags/models/tokenizer/',
        'model_weights' : [f'../input/uspppm-weights-2/USPPM-deberta-large-mnli-XE-NA-2.1-175-tags/models/model_{fold}.pth' for fold in range(5)],
        'fold_weights' : [0.85338,0.85016,0.85061,0.85099,0.84766],
        'max_len' : 175,
        'version' : 2.1,
        'directory' : 2
    },
    "USPPM-deberta-large-mnli-BCC-MSE-1-175-tags": {
        'config' : '../input/huggingface-models/deberta-large-mnli/config.json',
        'tokenizer' : '../input/uspppm-weights-2/USPPM-deberta-large-mnli-BCC-MSE-1-175-tags/models/tokenizer/',
        'model_weights' : [f'../input/uspppm-weights-2/USPPM-deberta-large-mnli-BCC-MSE-1-175-tags/models/model_{fold}.pth' for fold in range(5)],
        'fold_weights' : [0.85338,0.85016,0.85061,0.85099,0.84766],
        'max_len' : 175,
        'version' : 1,
        'directory' : 2
    },
    "USPPM-deberta-v3-large-BCC-MSE-1-133-tags": {
        'config' : '../input/huggingface-models/deberta-v3-large/config.json',
        'tokenizer' : '../input/uspppm-weights-2/USPPM-deberta-v3-large-BCC-MSE-1-133-tags/models/tokenizer/',
        'model_weights' : [f'../input/uspppm-weights-2/USPPM-deberta-v3-large-BCC-MSE-1-133-tags/models/model_{fold}.pth' for fold in range(5)],
        'fold_weights' : [0.86796,0.86079,0.86377,0.86062,0.86571],
        'max_len' : 133,
        'version' : 1,
        'directory' : 2
    },
    # "USPPM-bert-for-patents-BCC-MSE-1-117-tags": {
    #     'config' : '../input/huggingface-models/bert-for-patents/config.json',
    #     'tokenizer' : '../input/uspppm-weights-2/USPPM-bert-for-patents-BCC-MSE-1-117-tags/models/tokenizer/',
    #     'model_weights' : [f'../input/uspppm-weights-2/USPPM-bert-for-patents-BCC-MSE-1-117-tags/models/model_{fold}.pth' for fold in range(5)],
    #     'fold_weights' : [0.86013,0.84985,0.850340,0.85228,0.85214],
    #     'max_len' : 117,
    #     'version' : 1,
    #     'directory' : 2
    # },   
    
}

In [22]:
oof_0_files = [f'''/content/uspppm-weights-{value["directory"]}/{key}/models/oof_0.csv''' for key, value in model_dict.items()] 
oof_1_files = [f'''/content/uspppm-weights-{value["directory"]}/{key}/models/oof_1.csv''' for key, value in model_dict.items()] 
oof_2_files = [f'''/content/uspppm-weights-{value["directory"]}/{key}/models/oof_2.csv''' for key, value in model_dict.items()]
oof_3_files = [f'''/content/uspppm-weights-{value["directory"]}/{key}/models/oof_3.csv''' for key, value in model_dict.items()]
oof_4_files = [f'''/content/uspppm-weights-{value["directory"]}/{key}/models/oof_4.csv''' for key, value in model_dict.items()]

In [23]:
oof_dfs = dict()
oof_dfs[0] = pd.concat([pd.read_csv(file)['preds'] for file in oof_0_files], axis= 1)
oof_dfs[1] = pd.concat([pd.read_csv(file)['preds'] for file in oof_1_files], axis= 1)
oof_dfs[2] = pd.concat([pd.read_csv(file)['preds'] for file in oof_2_files], axis= 1)
oof_dfs[3] = pd.concat([pd.read_csv(file)['preds'] for file in oof_3_files], axis= 1)
oof_dfs[4] = pd.concat([pd.read_csv(file)['preds'] for file in oof_4_files], axis= 1)

In [24]:
for i in range(5):
    oof_dfs[i]['fold_'] = i

In [25]:
oof_df = pd.concat([value for key, value in oof_dfs.items()], axis = 0).reset_index(drop = True)
oof_df.columns = [f'preds_{i}' for i in range(oof_df.shape[1]-1)] + ['fold_']
oof_df.head()

,preds_0,preds_1,preds_2,preds_3,preds_4,preds_5,preds_6,preds_7,fold_
0,0.452213,0.007240,0.448309,0.003868,0.001913,0.080379,0.451749,0.445373,0
1,0.521835,0.247962,0.522612,0.252367,0.247341,0.244166,0.529531,0.502441,0
2,0.593666,0.035978,0.459657,0.011366,0.004027,0.490629,0.571922,0.501710,0
3,0.572030,0.013330,0.445843,0.002035,0.001615,0.500940,0.568172,0.454373,0
4,0.543219,0.252765,0.585853,0.323526,0.258561,0.394313,0.568515,0.548663,0


In [26]:
val_dfs = []
for fold in CFG.trn_fold:
    val_df = train[train["fold"] == fold].reset_index(drop = True)    
    val_oof  = oof_df[oof_df["fold_"] == fold].reset_index(drop = True)
    val_dfs.append(pd.concat([val_df, val_oof], axis =1))
train = pd.concat(val_dfs, axis = 0).reset_index(drop = True)            

In [27]:
train.head()

,id,anchor,target,context,score,context_text,context_tags,context_num,text,score_map,...,fold,preds_0,preds_1,preds_2,preds_3,preds_4,preds_5,preds_6,preds_7,fold_
0,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,<A47>,10,abatement forest region,0,...,0,0.452213,0.007240,0.448309,0.003868,0.001913,0.080379,0.451749,0.445373,0
1,ef2d4c2e6bbb208d,abatement,mixing core materials,A47,0.25,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,<A47>,10,abatement mixing core materials,1,...,0,0.521835,0.247962,0.522612,0.252367,0.247341,0.244166,0.529531,0.502441,0
2,cc96541d4987b399,abatement,rent abatement,A47,0.00,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,<A47>,10,abatement rent abatement,0,...,0,0.593666,0.035978,0.459657,0.011366,0.004027,0.490629,0.571922,0.501710,0
3,a8c9e9f37d4d836a,abatement,tax abatement,A47,0.00,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,<A47>,10,abatement tax abatement,0,...,0,0.572030,0.013330,0.445843,0.002035,0.001615,0.500940,0.568172,0.454373,0
4,87af8ea095cc18d4,abatement,prevention,A61,0.50,HUMAN NECESSITIES. MEDICAL OR VETERINARY SCIEN...,<A61>,11,abatement prevention,2,...,0,0.543219,0.252765,0.585853,0.323526,0.258561,0.394313,0.568515,0.548663,0


In [28]:
xgb_params = {
    "max_depth" : 9,
    "tree_method" : "gpu_hist",
    "eta" : 0.1,
    "subsample" : 1.0,
    "grow_policy" : "depthwise"    
}

In [29]:
ft_model = ft.load_model("/content/crawl-300d-2M-subword.bin")

In [30]:
def objective(trial):
    param = {        
        "device": "gpu",
        "gpu_platform_id": 0,
        "gpu_device_id": 0,
        "n_estimators": trial.suggest_int('n_estimators', 7000, 10000, step = 50),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "num_leaves": trial.suggest_int("num_leaves", 20, 3000, step=20),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 200, 10000, step=100),
        "lambda_l1": trial.suggest_int("lambda_l1", 0, 100, step=5),
        "lambda_l2": trial.suggest_int("lambda_l2", 0, 100, step=5),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0, 15),
        "bagging_fraction": trial.suggest_float(
            "bagging_fraction", 0.2, 0.95, step=0.1
        ),
        "bagging_freq": trial.suggest_categorical("bagging_freq", [1]),
        "feature_fraction": trial.suggest_float(
            "feature_fraction", 0.2, 0.95, step=0.1
        ),
        "verbose": -1,
        "n_jobs": -1
    }
    return(return_score(param))

In [31]:
def return_score(params):
    scores = []
    for fold in CFG.trn_fold:
        train_df = train[train["fold"]!=fold]
        val_df = train[train["fold"] == fold]    
        train_text_embeddings = np.array([ft_model.get_sentence_vector(text) for text in list(train_df.text)])
        val_text_embeddings = np.array([ft_model.get_sentence_vector(text) for text in list(val_df.text)])
        train_x = np.concatenate([train_df[['context_num'] + [f'preds_{i}' for i in range(oof_df.shape[1]-1)]].values, train_text_embeddings], axis = -1)
        val_x = np.concatenate([val_df[['context_num'] + [f'preds_{i}' for i in range(oof_df.shape[1]-1)]].values, val_text_embeddings], axis = -1)
        model = lgbm.LGBMRegressor(**params)
        model.fit(X = train_x, y = train_df["score"].values)
        score = scoring(np.array(val_df.score.values), np.array(model.predict(val_x)))
        scores.append(score)
    return np.mean(scores)

In [32]:
# study = optuna.create_study(direction='maximize',sampler=TPESampler())
# study.optimize(objective, n_trials= 120,show_progress_bar = True)

In [33]:
best_params = {'n_estimators': 8000, 'learning_rate': 0.17652113553748425, 'num_leaves': 1640, 'max_depth': 12, 'min_data_in_leaf': 800, 'lambda_l1': 0, 'lambda_l2': 65, 'min_gain_to_split': 1.2663080974789533, 'bagging_fraction': 0.9, 'bagging_freq': 1, 'feature_fraction': 0.2}

In [34]:
! mkdir "/content/drive/MyDrive/Top 9 LGBM/"

In [36]:
models = list()
scores =list()
for fold in CFG.trn_fold:
    train_df = train[train["fold"]!=fold]
    val_df = train[train["fold"] == fold]    
    train_text_embeddings = np.array([ft_model.get_sentence_vector(text) for text in list(train_df.text)])
    val_text_embeddings = np.array([ft_model.get_sentence_vector(text) for text in list(val_df.text)])
    train_x = np.concatenate([train_df[['context_num'] + [f'preds_{i}' for i in range(oof_df.shape[1]-1)]].values, train_text_embeddings], axis = -1)
    val_x = np.concatenate([val_df[['context_num'] + [f'preds_{i}' for i in range(oof_df.shape[1]-1)]].values, val_text_embeddings], axis = -1)
    model = lgbm.LGBMRegressor(**best_params)
    model.fit(X = train_x, y = train_df["score"].values)
    score = scoring(np.array(val_df.score.values), np.array(model.predict(val_x)))
    scores.append(score)
    print(f"Fold {fold} score: {score:.5f}")
    model.booster_.save_model(f"/content/drive/MyDrive/Top 9 LGBM/lgbm_{fold}_{score:.5f}.txt")
print(f"Mean score: {np.mean(scores)}")

Streaming output truncated to the last 5000 lines.
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there ar